In [15]:
from cleaning import data_new, X_train, X_test, y_train, y_test,numeric_columns,X,y

In [16]:
# Import necessary libraries and modules
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.decomposition import PCA
from nb import NaiveBayesClassifier
import matplotlib.pyplot as plt
# Define degree for polynomial features (adjust as needed)
degree = 2
poly = PolynomialFeatures(degree=degree)

# Define imputation and transformation strategies
numeric_transformer = SimpleImputer(strategy='mean')
categorical_transformer = SimpleImputer(strategy='most_frequent')
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Create ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Pipeline for Gaussian Naive Bayes without PCA
pipeline_without_pca = Pipeline([
    ('preprocessor', preprocessor),
    ('clf',NaiveBayesClassifier())
])

# Fit the pipeline to the training data
pipeline_without_pca.fit(X_train, y_train)

# Use the pipeline to predict on the test data
y_pred_without_pca = pipeline_without_pca.predict(X_test)

# Calculate accuracy and print classification report for without PCA
accuracy_without_pca = accuracy_score(y_test, y_pred_without_pca)
print("Accuracy without PCA:", accuracy_without_pca)
print("Classification Report without PCA:")
print(classification_report(y_test, y_pred_without_pca))

# Steps for Gaussian Naive Bayes with PCA
# Step 1: Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Step 2: Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)
X_scaled = scaler.fit_transform(X)

# Step 3: Apply PCA
pca = PCA(n_components=0.93)  # Adjust the variance as needed
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
X_pca = pca.fit_transform(X_scaled)
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(pca.explained_variance_) + 1), pca.explained_variance_, marker='o')
plt.title('Scree Plot')
plt.xlabel('Number of Principal Components')
plt.ylabel('Explained Variance')
plt.show()

# Explained variance ratio plot
plt.figure(figsize=(8, 6))
plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
plt.title('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.ylabel('Variance Ratio')
plt.show()

# Scatter plot of first two principal components
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=data.target, cmap='viridis')
plt.title('Scatter Plot of Principal Components')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar(label='Species')
plt.show()
# Step 4: Train and evaluate Gaussian Naive Bayes with PCA
gnb_pca = GaussianNB()
gnb_pca.fit(X_train_pca, y_train)
y_pred_pca = gnb_pca.predict(X_test_pca)

# Calculate accuracy and print classification report for with PCA
accuracy_with_pca = accuracy_score(y_test, y_pred_pca)
print("Accuracy with PCA:", accuracy_with_pca)
print("Classification Report with PCA:")
print(classification_report(y_test, y_pred_pca))


/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Accuracy without PCA: 0.4090909090909091
Classification Report without PCA:
              precision    recall  f1-score   support

           0       0.45      0.23      0.30       136
           1       0.39      0.64      0.49       106

    accuracy                           0.41       242
   macro avg       0.42      0.43      0.39       242
weighted avg       0.42      0.41      0.38       242



/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


ValueError: could not convert string to float: 'mr'

In [4]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

class RandomClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, random_state=None):
        """Initialize the classifier.
        
        Args:
            random_state (int, optional): Random state for reproducibility. Defaults to None.
        """
        self.random_state = random_state
        self.classes_ = None
        
    def fit(self, X, y):
        """Fit the classifier (learn the classes).
        
        Args:
            X (array-like): Training data (features).
            y (array-like): Target labels (classes).
        
        Returns:
            self: The fitted model.
        """
        # Store the unique classes in the training data
        self.classes_ = np.unique(y)
        
        # Set random seed for reproducibility (if provided)
        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        # Return self to indicate the fit is complete
        return self
        
    def predict(self, X):
        """Predict class labels randomly.
        
        Args:
            X (array-like): Test data (features).
        
        Returns:
            array: Predicted class labels.
        """
        # Predict random classes based on the learned classes
        random_indices = np.random.randint(len(self.classes_), size=len(X))
        random_predictions = self.classes_[random_indices]
        
        return random_predictions
    
    def get_params(self, deep=True):
        """Get parameters for the estimator.
        
        Args:
            deep (bool, optional): Whether to get parameters recursively. Defaults to True.
        
        Returns:
            dict: Dictionary of parameters.
        """
        return {"random_state": self.random_state}
    
    def set_params(self, **params):
        """Set parameters for the estimator.
        
        Args:
            **params: Named parameters to set.
        
        Returns:
            self: The updated model.
        """
        for param, value in params.items():
            setattr(self, param, value)
        
        return self


In [6]:
# Import necessary modules
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

# Handle missing values and categorical data with ColumnTransformer
numeric_features = X_train.columns.difference(['civilityTitle'])  # Update with your actual column names
categorical_features = ['civilityTitle']  # Update with your actual column names

numeric_transformer = SimpleImputer(strategy='mean')  # Use mean imputation for numeric data
categorical_transformer = SimpleImputer(strategy='most_frequent')  # Use most frequent for categorical data

# Create ColumnTransformer with both transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define a pipeline with the preprocessor and PCA
pipeline_pca = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=100)),
    ('clf', RandomClassifier(random_state=42))
])

# Train the pipeline with PCA on the training data
pipeline_pca.fit(X_train, y_train)

# Predict on the testing data using the pipeline
y_pred_pca = pipeline_pca.predict(X_test)

# Calculate accuracy and print classification report
accuracy_pca = accuracy_score(y_test, y_pred_pca)
report_pca = classification_report(y_test, y_pred_pca)

print("\nRandom Forest with PCA:")
print("Accuracy:", accuracy_pca)
print(report_pca)

# Define a second pipeline without PCA for comparison
pipeline_original = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', RandomClassifier(random_state=42))
])

# Train the pipeline without PCA on the training data
pipeline_original.fit(X_train, y_train)

# Predict on the testing data using the pipeline
y_pred_original = pipeline_original.predict(X_test)

# Calculate accuracy and print classification report
accuracy_original = accuracy_score(y_test, y_pred_original)
report_original = classification_report(y_test, y_pred_original)

print("\nRandom Forest without PCA:")
print("Accuracy:", accuracy_original)
print(report_original)


/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(



Random Forest with PCA:
Accuracy: 0.5702479338842975
              precision    recall  f1-score   support

           0       0.63      0.56      0.59       136
           1       0.51      0.58      0.54       106

    accuracy                           0.57       242
   macro avg       0.57      0.57      0.57       242
weighted avg       0.58      0.57      0.57       242


Random Forest without PCA:
Accuracy: 0.5702479338842975
              precision    recall  f1-score   support

           0       0.63      0.56      0.59       136
           1       0.51      0.58      0.54       106

    accuracy                           0.57       242
   macro avg       0.57      0.57      0.57       242
weighted avg       0.58      0.57      0.57       242



/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# Define a pipeline with the preprocessor, scaler, PCA, and Logistic Regression
pipeline_pca_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.80)),  # Adjust n_components as needed
    ('clf', LogisticRegression(max_iter=1000))
])

# Train the pipeline with PCA on the training data
pipeline_pca_lr.fit(X_train, y_train)

# Predict on the testing data using the pipeline
y_pred_pca_lr = pipeline_pca_lr.predict(X_test)

# Calculate accuracy and print classification report
accuracy_pca_lr = accuracy_score(y_test, y_pred_pca_lr)
report_pca_lr = classification_report(y_test, y_pred_pca_lr)

print("\nLogistic Regression with PCA:")
print("Accuracy:", accuracy_pca_lr)
print(report_pca_lr)

# Define a second pipeline without PCA for comparison
pipeline_original_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000))
])

# Train the pipeline without PCA on the training data
pipeline_original_lr.fit(X_train, y_train)

# Predict on the testing data using the pipeline
y_pred_original_lr = pipeline_original_lr.predict(X_test)

# Calculate accuracy and print classification report
accuracy_original_lr = accuracy_score(y_test, y_pred_original_lr)
report_original_lr = classification_report(y_test, y_pred_original_lr)

print("\nLogistic Regression without PCA:")
print("Accuracy:", accuracy_original_lr)
print(report_original_lr)


/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(



Logistic Regression with PCA:
Accuracy: 0.5950413223140496
              precision    recall  f1-score   support

           0       0.61      0.75      0.68       136
           1       0.55      0.40      0.46       106

    accuracy                           0.60       242
   macro avg       0.58      0.57      0.57       242
weighted avg       0.59      0.60      0.58       242


Logistic Regression without PCA:
Accuracy: 0.5867768595041323
              precision    recall  f1-score   support

           0       0.61      0.73      0.66       136
           1       0.54      0.41      0.46       106

    accuracy                           0.59       242
   macro avg       0.57      0.57      0.56       242
weighted avg       0.58      0.59      0.58       242



/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


In [14]:
from sklearn.neighbors import KNeighborsClassifier
# Define a pipeline with the preprocessor, scaler, PCA, and K-Nearest Neighbors
pipeline_pca_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=100)),  # Adjust n_components as needed
    ('clf', KNeighborsClassifier())
])

# Train the pipeline with PCA on the training data
pipeline_pca_knn.fit(X_train, y_train)

# Predict on the testing data using the pipeline
y_pred_pca_knn = pipeline_pca_knn.predict(X_test)

# Calculate accuracy and print classification report
accuracy_pca_knn = accuracy_score(y_test, y_pred_pca_knn)
report_pca_knn = classification_report(y_test, y_pred_pca_knn)

print("\nK-Nearest Neighbors with PCA:")
print("Accuracy:", accuracy_pca_knn)
print(report_pca_knn)

# Define a second pipeline without PCA for comparison
pipeline_original_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier())
])

# Train the pipeline without PCA on the training data
pipeline_original_knn.fit(X_train, y_train)

# Predict on the testing data using the pipeline
y_pred_original_knn = pipeline_original_knn.predict(X_test)

# Calculate accuracy and print classification report
accuracy_original_knn = accuracy_score(y_test, y_pred_original_knn)
report_original_knn = classification_report(y_test, y_pred_original_knn)

print("\nK-Nearest Neighbors without PCA:")
print("Accuracy:", accuracy_original_knn)
print(report_original_knn)


/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(



K-Nearest Neighbors with PCA:
Accuracy: 0.6033057851239669
              precision    recall  f1-score   support

           0       0.65      0.63      0.64       136
           1       0.55      0.57      0.56       106

    accuracy                           0.60       242
   macro avg       0.60      0.60      0.60       242
weighted avg       0.61      0.60      0.60       242


K-Nearest Neighbors without PCA:
Accuracy: 0.6033057851239669
              precision    recall  f1-score   support

           0       0.65      0.63      0.64       136
           1       0.55      0.57      0.56       106

    accuracy                           0.60       242
   macro avg       0.60      0.60      0.60       242
weighted avg       0.61      0.60      0.60       242



/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
/home/harsha/.local/lib/python3.10/site-packages/sklearn/impute/_base.py:577: UserWarning: Skipping features without any observed values: ['civilityTitle']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
